In [0]:
spark

####Create Dataframe

In [0]:

my_data = [( 1,   1),   
( 2,   1),   
( 3,   1),  
( 4,   2),   
( 5,   1),   
( 6,   2),   
( 7,   2),]  

In [0]:
my_schema = ['id', 'num']

In [0]:
# from pyspark.sql.types import StructField, StructType

# my_schema = StructType([
#     StructField('id', IntegerType(), True)
# ])

In [0]:
spark.createDataFrame(data=my_data, schema = my_schema).show()

####Transformations on Dataframe


In [0]:
df.printSchema()
df.columns

In [0]:
from pyspark.sql import Row

In [0]:
row = Row(1, 'Addy', 21, 40000)

In [0]:
%fs

ls /FileStore/tables/

In [0]:
employee_df = spark.read.format('csv')\
    .option('header','true')\
    .option('inferschema','true')\
    .option('mode','PERMISSIBLE')\
    .load('dbfs:/FileStore/tables/emp_data.csv')

employee_df.show()

In [0]:
employee_df.printSchema()


#####Different SELECT options

In [0]:
employee_df.select('age').show()

In [0]:
from pyspark.sql.functions import *

In [0]:
employee_df.select(col('age'), 'name').show()

In [0]:
employee_df.select(col('id')+5).show()


In [0]:
employee_df.select(expr('id+5')).show()

In [0]:
employee_df.columns

#####SELECT multiple columns

In [0]:
employee_df.select('id',col('name'),employee_df['age'], employee_df.address).show()

In [0]:
employee_df.select(expr('id as IdentifierId'),expr('name as EmployeeName'), expr('address as EmpAddress'), expr('concat(name,address) as Concat')).show()

In [0]:
from pyspark.sql.functions import *

In [0]:
employee_df.select(col('id').alias('employee_id'),col('name').alias('employee_name'),'age').show()

In [0]:
employee_df.select('*').show()

####Spark SQL

In [0]:
employee_df = spark.read.format('csv')\
    .option('header','true')\
    .option('inferschema','true')\
    .option('mode','PERMISSIBLE')\
    .load('dbfs:/FileStore/tables/emp_data.csv')

employee_df.show()

In [0]:
employee_df.createOrReplaceTempView('employee_table')

In [0]:
spark.sql("""

    select * from employee_table

""").show()

In [0]:
spark.sql("""
          
    select id as Employee_id, name, age from employee_table

""").show()

####Transformations

In [0]:
employee_df = spark.read.format('csv')\
    .option('header','true')\
    .option('inferschema','true')\
    .option('mode','PERMISSIBLE')\
    .load('dbfs:/FileStore/tables/emp_data.csv')

employee_df.show()

#####Aliasing

In [0]:
employee_df.select(col('id').alias('employee_id'),col('name').alias('employee_name'),'age').show()

#####filter/where - both are same

In [0]:
employee_df.filter(col('age')>22).show()

In [0]:
employee_df.where(col('age')>22).show()

In [0]:
employee_df.filter(col('salary')>150000 & col('age')>25).show()

In [0]:
employee_df.filter((col('salary')>150000) & (col('age')>=25)).show()

#####literal

In [0]:
employee_df = employee_df.select('*',lit('Arora').alias('Last_Name'))
employee_df.show()


#####Adding Columns - **WithColumn** keyword - can add new columns or can overwrite if column is already present.

In [0]:
employee_df = employee_df.withColumn('sur_name',lit('khan').cast('string'))
employee_df.show()

In [0]:
employee_df.show()


#####rename Columns

In [0]:
employee_df.withColumnRenamed('id','employee_id').show()


#####cast data types

In [0]:
employee_df.printSchema()

In [0]:
employee_df.withColumn('id',col('id').cast('string'))\
        .withColumn('salary', col('salary').cast('long'))\
        .printSchema()


#####deleting column

In [0]:
employee_df = employee_df.drop('Last_Name',col('sur_name'))
employee_df.show()

####Spark SQL

In [0]:
employee_df.createOrReplaceTempView('table')

In [0]:
spark.sql("""
          
    SELECT id as Employee_Id, age, name as Employee_Name, CAST(salary as FLOAT), 'FamilyHero' as Title, concat(name,'-',Title,'-',salary) as SubTitle from table
    WHERE salary > 150000 OR Age >=25

""").show(truncate=False)

####Union & unionAll

**In SQL or Spark SQL<br>**
Union - when done between two entities generates output with no same/duplicate records.<br>
UnionAll - when done between two entities generates output which allows duplicate records as well.<br>

**In Dataframe.**<br> Both Union and UnionAll gives same result and count of recrods allowing the duplicate records.


In [0]:
data=[(10 ,'Anil',50000, 18),
(11 ,'Vikas',75000,  16),
(12 ,'Nisha',40000,  18),
(13 ,'Nidhi',60000,  17),
(14 ,'Priya',80000,  18),
(15 ,'Mohit',45000,  18),
(16 ,'Rajesh',90000, 10),
(17 ,'Raman',55000, 16),
(18 ,'Sam',65000,   17)]


schema = ['id','Name','salary','manager_id']

manager_df = spark.createDataFrame(data = data, schema = schema)
          
        

In [0]:
manager_df.show()

In [0]:
manager_df.count()  #count gives gives number of records

In [0]:
data1=[(19 ,'Sohan',50000, 18),
(20 ,'Sima',75000,  17)]

schema1=['id','Name','salary','manager_id']

manager_df1 = spark.createDataFrame(data= data1, schema=schema1)

In [0]:
manager_df1.show()

In [0]:
manager_df.union(manager_df1).show()

In [0]:
manager_df.union(manager_df1).count()

In [0]:
#duplicate data

data=[(10 ,'Anil',50000, 18),
(11 ,'Vikas',75000,  16),
(12 ,'Nisha',40000,  18),
(13 ,'Nidhi',60000,  17),
(14 ,'Priya',80000,  18),
(15 ,'Mohit',45000,  18),
(16 ,'Rajesh',90000, 10),
(17 ,'Raman',55000, 16),
(18 ,'Sam',65000,   17),
(18 ,'Sam',65000,   17),
(18 ,'Sam',50000,   17),
(19 ,'Assam',60000, 20)]

schema = ['id','name','sal','manager_id']


duplicate_df = spark.createDataFrame(data=data, schema=schema)
duplicate_df.show()

In [0]:
print(duplicate_df.count())
print(manager_df1.count())

In [0]:
duplicate_df.union(manager_df1).show()

In [0]:
duplicate_df.union(manager_df1).count()

In [0]:
duplicate_df.unionAll(manager_df1).show()
print(duplicate_df.unionAll(manager_df1).count())


Hence, we got same result and count of records for Union and UnionAll in Dataframe<br>Now let's check below for Spark SQL

In [0]:
duplicate_df.createOrReplaceTempView('duplicate_df')
manager_df1.createOrReplaceTempView('manager_df1')

In [0]:
query_output = spark.sql("""

  SELECT * FROM duplicate_df       
  UNION
  SELECT * FROM manager_df1

""")

In [0]:
query_output.show()
query_output.count()

In [0]:
unionAll_query_output = spark.sql("""

  SELECT * FROM duplicate_df       
  UNION ALL
  SELECT * FROM manager_df1

""")

In [0]:
unionAll_query_output.show()
unionAll_query_output.count() #giving all the records having duplicates too

In [0]:
data=[(10 ,'Anil',50000, 18),
(11 ,'Vikas',75000,  16),
(12 ,'Nisha',40000,  18),
(13 ,'Nidhi',60000,  17),
(14 ,'Priya',80000,  18),
(15 ,'Mohit',45000,  18),
(16 ,'Rajesh',90000, 10),
(17 ,'Raman',55000, 16),
(18 ,'Sam',65000,   17)]


schema = ['id','Name','salary','manager_id']

manager_df = spark.createDataFrame(data = data, schema = schema)
manager_df.show()
print(manager_df.count())          
manager_df.createOrReplaceTempView('manager_df')      

In [0]:
udf = spark.sql("""

    SELECT * from manager_df
    UNION
    SELECT* from manager_df

""")
udf.show()
udf.count()

In [0]:
wrong_column_data=[(19 ,50000, 18,'Sohan'),
(20 ,75000,  17,'Sima')]

wrong_schema = ['id','salary','manager_id', 'Name']

wrong_manager_df = spark.createDataFrame(data=wrong_column_data, schema = wrong_schema )


# wrong_column_data=[(19 ,50000, 18,'Sohan',10),
# (20 ,75000,  17,'Sima',20)]

# wrong_schema = ['id','sal','mngr_id', 'Name','bonus']



In [0]:
wrong_manager_df.show()
manager_df1.show()

In [0]:
manager_df1.union(wrong_manager_df).show()

####UnionByName

In [0]:
manager_df1.unionByName(wrong_manager_df).show()

above gives output only when column names are same. But what if the column names are different. We get an error

In [0]:
wrong_column_data=[(19 ,50000, 18,'Sohan'),
(20 ,75000,  17,'Sima')]

wrong_schema = ['id','slry','mngr_id', 'Name']

wrong_manager_df = spark.createDataFrame(data=wrong_column_data, schema = wrong_schema )

In [0]:
wrong_manager_df.show()
manager_df1.show()   #both have some diff. column names

In [0]:
manager_df1.unionByName(wrong_manager_df).show()

In [0]:
from pyspark.sql.functions import *

if extra columns are there, then select only what are required to apply union on

In [0]:
wrong_column_data=[(19 ,50000, 18,'Sohan',10),
(20 ,75000,  17,'Sima',20)]

wrong_schema = ['id','salary','manager_id', 'Name','bonus']

wrong_employee_df = spark.createDataFrame(data=wrong_column_data, schema=wrong_schema)

wrong_employee_df.show()
manager_df1.show()


In [0]:
wrong_employee_df.union(manager_df1).show()

In [0]:
wrong_employee_df.select(col('id'), col('name'), col('manager_id'),col('salary')).union(manager_df1).show()

In [0]:
wrong_employee_df.select(col('id'), col('name'), col('manager_id'),col('salary')).unionByName(manager_df1).show()

df1.unionByName(df2, **allowMissingColumns = True**)

In [0]:
wrong_employee_df.unionByName(manager_df1, allowMissingColumns=True).show()

####If Else in pySpark

#####when otherwise

In [0]:
emp_data = [
(1,'manish',26,20000,'india','IT'),
(2,'rahul',None,40000,'germany','engineering'),
(3,'pawan',12,60000,'india','sales'),
(4,'roshini',44,None,'uk','engineering'),
(5,'raushan',35,70000,'india','sales'),
(6,None,29,200000,'uk','IT'),
(7,'adam',37,65000,'us','IT'),
(8,'chris',16,40000,'us','sales'),
(None,None,None,None,None,None),
(7,'adam',37,65000,'us','IT')
]

emp_schema = ['id','name','age','salary','country','dept']

emp_df = spark.createDataFrame(data = emp_data, schema = emp_schema)

In [0]:
emp_df.show()

In [0]:
from pyspark.sql.functions import *

Add column and its value based on the some condition on other column. 

In [0]:
emp_df.withColumn('Adult', when(col('age')<18,'No').when(col('age')>18, 'Yes').otherwise("Novalue")).show()

replacing 'null' values in a column with other value.

In [0]:
emp_df.withColumn('age',when(col('age').isNull(),lit(19)).otherwise(col('age')))\
    .withColumn('adult', when(col('age')>18, 'Yes').otherwise("No")).show()

In [0]:
emp_df.createOrReplaceTempView('emp_df')

#####Using CASE WHEN in SPARK SQL

In [0]:
spark.sql("""
          
    SELECT *,
        CASE
            WHEN age<18 THEN 'Minor'
            WHEN age>18 THEN 'Major'
            ELSE 'Old'
        END AS Adult
        FROM emp_df
          """).show()


####Unique and Sorted records

In [0]:
data=[(10 ,'Anil',50000, 18),
(11 ,'Vikas',75000,  16),
(12 ,'Nisha',40000,  18),
(13 ,'Nidhi',60000,  17),
(14 ,'Priya',80000,  18),
(15 ,'Mohit',45000,  18),
(16 ,'Rajesh',90000, 10),
(17 ,'Raman',55000, 16),
(18 ,'Sam',65000,   17),
(15 ,'Mohit',45000,  18),
(13 ,'Nidhi',60000,  17),      
(14 ,'Priya',90000,  18),  
(18 ,'Sam',65000,   17)
]
schema = ['id','name','salary','manager_id']

employee_df = spark.createDataFrame(data=data, schema=schema)
employee_df.show()

#####Unique Rows (distinct)

In [0]:
print(employee_df.distinct().count())
print(employee_df.count())

In [0]:
employee_df.select('id','name').distinct().show()
employee_df.select('id','name').distinct().count()

#####Drop Dupplicate rows

In [0]:
employee_df.select('*').orderBy('name').show()

In [0]:
employee_df.dropDuplicates().show()

In [0]:
employee_df.dropDuplicates(['id','name']).show()

#####Sort in Ascending/descending

In [0]:
employee_df.sort('salary').asc().show()

In [0]:
from pyspark.sql.functions import *

In [0]:
employee_df.sort(col('salary').desc()).show()

In [0]:
employee_df.sort(col('salary').desc(), col('name').desc()).show()

In [0]:
employee_df.dropDuplicates().sort(col('salary').desc(), col('name').desc()).show()

####LeetCode Question

In [0]:
leet_code_data = [
    (1, 'Will', None),
    (2, 'Jane', None),
    (3, 'Alex', 2),
    (4, 'Bill', None),
    (5, 'Zack', 1),
    (6, 'Mark', 2)
]
schema = ['id','name','refree_id']

df = spark.createDataFrame(data=leet_code_data, schema=schema)
df.show()

In [0]:
filtered_df = df.select('name').filter((col('refree_id')!=2) | (col('refree_id').isNull()))
filtered_df.show()

####Aggregate Functions

#####count, min, max, sum, avg

In [0]:
emp_data = [
(1,'manish',26,20000,'india','IT'),
(2,'rahul',None,40000,'germany','engineering'),
(3,'pawan',12,60000,'india','sales'),
(4,'roshini',44,None,'uk','engineering'),
(5,'raushan',35,70000,'india','sales'),
(6,None,29,200000,'uk','IT'),
(7,'adam',37,65000,'us','IT'),
(8,'chris',16,40000,'us','sales'),
(None,None,None,None,None,None),
(7,'adam',37,65000,'us','IT')
]

my_schema = ['id','name','age','salary','country','dept']

employee_df = spark.createDataFrame(data=emp_data, schema=my_schema)
employee_df.show()

In [0]:
employee_df.count()  #transfromation

In [0]:
employee_df.select(count('name')).show()  #upon single column - does not count null.
employee_df.select(count('id')).show()

In [0]:
employee_df.count() #count is an action here. Job is created.

In [0]:
employee_df.select(count('country'))  #count is a transformation here. It will be lazly evaluated, until a job is created using an action.

In [0]:
employee_df.select(count('*')).show() #funny behaviour of count. When count on all columns - gives null record count too.

In [0]:
employee_df.select(sum('salary').alias('Total_Salary'), min('salary'), max('salary').alias('Max_Salary')).show()

In [0]:
employee_df.select(sum('salary'), count('salary'), avg('salary').cast('int').alias('Average_Salary')).show()

In [0]:
employee_df.select((sum('salary')/count('salary')).alias('Average_Salary').cast('int')).show()

####Group By

In [0]:
emp_data=[(1,'manish',50000,'IT'),
(2,'vikash',60000,'sales'),
(3,'raushan',70000,'marketing'),
(4,'mukesh',80000,'IT'),
(5,'pritam',90000,'sales'),
(6,'nikita',45000,'marketing'),
(7,'ragini',55000,'marketing'),
(8,'rakesh',100000,'IT'),
(9,'aditya',65000,'IT'),
(10,'rahul',50000,'marketing')]

schema = ['id','name','salary','dept']

employee_df=spark.createDataFrame(data=emp_data, schema=schema)
employee_df.show()

In [0]:
employee_df.groupBy('dept').agg(sum('salary').alias('DepartmentWise_Salary')).show()


In [0]:
emp_data = [(1,'manish',50000,'IT','india'),
(2,'vikash',60000,'sales','us'),
(3,'raushan',70000,'marketing','india'),
(4,'mukesh',80000,'IT','us'),
(5,'pritam',90000,'sales','india'),
(6,'nikita',45000,'marketing','us'),
(7,'ragini',55000,'marketing','india'),
(8,'rakesh',100000,'IT','us'),
(9,'aditya',65000,'IT','india'),
(10,'rahul',50000,'marketing','us')]

schema = ['id','name','salary','dept','country']

emp_df=spark.createDataFrame(data=emp_data, schema=schema)
emp_df.show()

In [0]:
emp_df.printSchema()

In [0]:
from pyspark.sql.functions import *

In [0]:
emp_df.groupBy('dept','country').agg(sum('salary')).show()

In [0]:
employee_df.createOrReplaceTempView('employee_dff')

In [0]:
 spark.sql("""
           
    SELECT dept,sum(salary)
    FROM employee_dff
    GROUP BY dept

           """).show()

some Window Functions

In [0]:
spark.sql("""
          
 SELECT *,
  SUM(salary) OVER (PARTITION BY dept) AS New_col,
  ROUND(salary/New_col*100,2) AS percentage_of_empSalary
  FROM employee_dff
         
""").show()

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import *

In [0]:
emp_df.withColumn('New_col',sum('salary').over(Window.partitionBy('dept')))\
.withColumn('percentage_of_empSalary', round(col('salary')/col('New_col')*100,2)).select('*').show()

####Joins

In [0]:
customer_data = [(1,'manish','patna',"30-05-2022"),
(2,'vikash','kolkata',"12-03-2023"),
(3,'nikita','delhi',"25-06-2023"),
(4,'rahul','ranchi',"24-03-2023"),
(5,'mahesh','jaipur',"22-03-2023"),
(6,'prantosh','kolkata',"18-10-2022"),
(7,'raman','patna',"30-12-2022"),
(8,'prakash','ranchi',"24-02-2023"),
(9,'ragini','kolkata',"03-03-2023"),
(10,'raushan','jaipur',"05-02-2023")]

customer_schema=['customer_id','customer_name','address','date_of_joining']

customer_df = spark.createDataFrame(data=customer_data, schema=customer_schema)

sales_data = [(1,22,10,"01-06-2022"),
(1,27,5,"03-02-2023"),
(2,5,3,"01-06-2023"),
(5,22,1,"22-03-2023"),
(7,22,4,"03-02-2023"),
(9,5,6,"03-03-2023"),
(2,1,12,"15-06-2023"),
(1,56,2,"25-06-2023"),
(5,12,5,"15-04-2023"),
(11,12,76,"12-03-2023")]

sales_schema=['customer_id','product_id','quantity','date_of_purchase']

sales_df = spark.createDataFrame(data=sales_data, schema= sales_schema)

product_data = [(1, 'fanta',20),
(2, 'dew',22),
(5, 'sprite',40),
(7, 'redbull',100),
(12,'mazza',45),
(22,'coke',27),
(25,'limca',21),
(27,'pepsi',14),
(56,'sting',10)]

product_schema=['id','name','price']

product_df = spark.createDataFrame(data=product_data, schema=product_schema)

In [0]:
customer_df.show()

In [0]:
sales_df.show()

In [0]:
product_df.show()

In [0]:
customer_df.join(sales_df,customer_df['customer_id']==sales_df['customer_id'],'leftouter').show()

####Window Function

In [0]:
emp_data = [(1,'manish',50000,'IT','m'),
(2,'vikash',60000,'sales','m'),
(3,'raushan',70000,'marketing','m'),
(4,'mukesh',80000,'IT','m'),
(5,'priti',90000,'sales','f'),
(6,'nikita',45000,'marketing','f'),
(7,'ragini',55000,'marketing','f'),
(8,'rashi',100000,'IT','f'),
(9,'aditya',65000,'IT','m'),
(10,'rahul',50000,'marketing','m'),
(11,'rakhi',50000,'IT','f'),
(12,'akhilesh',90000,'sales','m')]

schema = ['id','Name','Salary','Department','Gender']

emp_df = spark.createDataFrame(data=emp_data, schema=schema)

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [0]:
emp_df.show()

In [0]:
window = Window.partitionBy('department','gender').orderBy(desc('salary'))
emp_df.withColumn('Dense_Rank',dense_rank().over(window))\
    .filter(col('Dense_Rank')<=2)\
    .show()

In [0]:
emp_df.withColumn('RowNumber',row_number().over(Window.partitionBy('Department').orderBy('Salary')))\
    .withColumn('Rank', rank().over(Window.partitionBy('Department').orderBy('Salary')))\
    .withColumn('DenseRank', dense_rank().over(Window.partitionBy('Department').orderBy('Salary')))\
    .show(truncate=False)


####Random:  to_date() 
--> Date-type convert

In [0]:
product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]
schema=['product_id','product_name','sales_date','sales']

prod_df=spark.createDataFrame(data=product_data,schema=schema)

prod_df.show()

In [0]:
prod_df.printSchema()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime

In [0]:
prod_df=prod_df.withColumn('sales_date',to_date(col('sales_date'),'dd-MM-yyyy'))
prod_df.printSchema()

In [0]:
prod_df.show()

In [0]:
prod_df.select(dayofmonth('sales_date').alias('day'),month(col('sales_date')).alias('month'),year('sales_date').alias('year')).show()

####Lead and Lag

In [0]:
product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]
schema=['product_id','product_name','sales_date','sales']

prod_df=spark.createDataFrame(data=product_data,schema=schema)

prod_df.show()

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [0]:
window = Window.partitionBy('product_id').orderBy('sales_date')
prod_df.withColumn('Lead_Data',lead('sales',2,101).over(window)).show()

In [0]:
window = Window.partitionBy('product_id').orderBy('sales_date')
prod_df.withColumn('Lag_data',lag('sales',2,101).over(window)).show()

In [0]:
prod_df.show()

In [0]:
prod_df.withColumn('sum6month',sum('sales').over(Window.partitionBy('product_id')))\
    .withColumn('percent',round(((col('sum6month')-col('sales'))/col('sum6month'))*100,2))\
    .show()

In [0]:
window=Window.partitionBy('product_id').orderBy('sales_date')
prod_df.withColumn('prev_month_%',lag(col('sales'),1,0).over(window))\
    .withColumn('%_sales_from prevMonth',round((col('sales')-col('prev_month_%'))/col('sales')*100,2)).show()

In [0]:
window=Window.partitionBy('product_id').orderBy('sales_date')
prod_df.withColumn('prev_month_%',round((col('sales')-lag(col('sales'),1,0).over(window))/col('sales')*100,2)).show()
   


#### First and Last
--> Rows Between and Range Between<br>
--> unboundedPreceding <br>
--> unboundedFollowing

In [0]:
emp_data = [(1,"manish","11-07-2023","10:20"),
        (1,"manish","11-07-2023","11:20"),
        (2,"rajesh","11-07-2023","11:20"),
        (1,"manish","11-07-2023","11:50"),
        (2,"rajesh","11-07-2023","13:20"),
        (1,"manish","11-07-2023","19:20"),
        (2,"rajesh","11-07-2023","17:20"),
        (1,"manish","12-07-2023","10:32"),
        (1,"manish","12-07-2023","12:20"),
        (3,"vikash","12-07-2023","09:12"),
        (1,"manish","12-07-2023","16:23"),
        (3,"vikash","12-07-2023","18:08")]

emp_schema = ["id", "name", "date", "time"]
emp_df = spark.createDataFrame(data=emp_data, schema=emp_schema)

emp_df.show()

+---+------+----------+-----+
| id|  name|      date| time|
+---+------+----------+-----+
|  1|manish|11-07-2023|10:20|
|  1|manish|11-07-2023|11:20|
|  2|rajesh|11-07-2023|11:20|
|  1|manish|11-07-2023|11:50|
|  2|rajesh|11-07-2023|13:20|
|  1|manish|11-07-2023|19:20|
|  2|rajesh|11-07-2023|17:20|
|  1|manish|12-07-2023|10:32|
|  1|manish|12-07-2023|12:20|
|  3|vikash|12-07-2023|09:12|
|  1|manish|12-07-2023|16:23|
|  3|vikash|12-07-2023|18:08|
+---+------+----------+-----+



In [0]:
emp_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)




Employee out for more than 8 Hours

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [0]:
window=Window.partitionBy('id','date').orderBy('time').rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)

emp_df.withColumn('ClockIn',first('time').over(window))\
    .withColumn('ClockOut',last('time').over(window)).show()

+---+------+----------+-----+-------+--------+
| id|  name|      date| time|ClockIn|ClockOut|
+---+------+----------+-----+-------+--------+
|  1|manish|11-07-2023|10:20|  10:20|   19:20|
|  1|manish|11-07-2023|11:20|  10:20|   19:20|
|  1|manish|11-07-2023|11:50|  10:20|   19:20|
|  1|manish|11-07-2023|19:20|  10:20|   19:20|
|  1|manish|12-07-2023|10:32|  10:32|   16:23|
|  1|manish|12-07-2023|12:20|  10:32|   16:23|
|  1|manish|12-07-2023|16:23|  10:32|   16:23|
|  2|rajesh|11-07-2023|11:20|  11:20|   17:20|
|  2|rajesh|11-07-2023|13:20|  11:20|   17:20|
|  2|rajesh|11-07-2023|17:20|  11:20|   17:20|
|  3|vikash|12-07-2023|09:12|  09:12|   18:08|
|  3|vikash|12-07-2023|18:08|  09:12|   18:08|
+---+------+----------+-----+-------+--------+



In [0]:
emp_df.withColumn('OfficeHours',first('time').over(window)-last('time').over(window)).show()

+---+------+----------+-----+-----------+
| id|  name|      date| time|OfficeHours|
+---+------+----------+-----+-----------+
|  1|manish|11-07-2023|10:20|       null|
|  1|manish|11-07-2023|11:20|       null|
|  1|manish|11-07-2023|11:50|       null|
|  1|manish|11-07-2023|19:20|       null|
|  1|manish|12-07-2023|10:32|       null|
|  1|manish|12-07-2023|12:20|       null|
|  1|manish|12-07-2023|16:23|       null|
|  2|rajesh|11-07-2023|11:20|       null|
|  2|rajesh|11-07-2023|13:20|       null|
|  2|rajesh|11-07-2023|17:20|       null|
|  3|vikash|12-07-2023|09:12|       null|
|  3|vikash|12-07-2023|18:08|       null|
+---+------+----------+-----+-----------+



In [0]:
from pyspark.sql import functions as F

In [0]:
  # .withColumn('OfficeHours',from_unixtime((unix_timestamp(col('ClockOut')) - unix_timestamp(col('ClockIn'))),'HH:mm'))  # Convert seconds to hours

from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col, to_timestamp, first, last, unix_timestamp, second



# Convert date and time columns to timestamp
#emp_df = emp_df.withColumn('date', to_timestamp('date', 'dd-MM-yyyy'))
emp_df = emp_df.withColumn('time', to_timestamp('time', 'HH:mm'))

# Window specification for each employee and date
window_spec = Window.partitionBy('id', 'date').orderBy('time').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

# Calculate ClockIn, ClockOut, and OfficeHours
result_df = emp_df.withColumn('ClockIn', first('time').over(window_spec))\
    .withColumn('ClockOut', last('time').over(window_spec))\
    .withColumn('OfficeHours', (unix_timestamp(col('ClockOut')) - unix_timestamp(col('ClockIn')))/3600)  # Convert seconds to hours

# Show the result
result_df.show()
result_df.printSchema()


+---+------+----------+-------------------+-------------------+-------------------+-----------------+
| id|  name|      date|               time|            ClockIn|           ClockOut|      OfficeHours|
+---+------+----------+-------------------+-------------------+-------------------+-----------------+
|  1|manish|11-07-2023|1970-01-01 10:20:00|1970-01-01 10:20:00|1970-01-01 19:20:00|              9.0|
|  1|manish|11-07-2023|1970-01-01 11:20:00|1970-01-01 10:20:00|1970-01-01 19:20:00|              9.0|
|  1|manish|11-07-2023|1970-01-01 11:50:00|1970-01-01 10:20:00|1970-01-01 19:20:00|              9.0|
|  1|manish|11-07-2023|1970-01-01 19:20:00|1970-01-01 10:20:00|1970-01-01 19:20:00|              9.0|
|  1|manish|12-07-2023|1970-01-01 10:32:00|1970-01-01 10:32:00|1970-01-01 16:23:00|             5.85|
|  1|manish|12-07-2023|1970-01-01 12:20:00|1970-01-01 10:32:00|1970-01-01 16:23:00|             5.85|
|  1|manish|12-07-2023|1970-01-01 16:23:00|1970-01-01 10:32:00|1970-01-01 16:23:00


(Running sum) Sales performance - last 3 months

In [0]:
product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]

product_schema=["product_id","product_name","sales_date","sales"]

product_df = spark.createDataFrame(data=product_data,schema=product_schema)

In [0]:
window = Window.partitionBy('product_id').orderBy('sales_date').rowsBetween(-2,0)

product_df.withColumn('running_sum',sum('sales').over(window))\
    .withColumn('avg_3months',round(col('running_sum')/3,2))\
    .filter(substring('sales_date',5,1)>2).show()#filtering out Jan and Feb month.


+----------+------------+----------+-------+-----------+-----------+
|product_id|product_name|sales_date|  sales|running_sum|avg_3months|
+----------+------------+----------+-------+-----------+-----------+
|         1|      iphone|01-03-2023|1600000|    4400000| 1466666.67|
|         1|      iphone|01-04-2023|1700000|    4600000| 1533333.33|
|         1|      iphone|01-05-2023|1200000|    4500000|  1500000.0|
|         1|      iphone|01-06-2023|1100000|    4000000| 1333333.33|
|         2|     samsung|01-03-2023|1080000|    3300000|  1100000.0|
|         2|     samsung|01-04-2023|1800000|    4000000| 1333333.33|
|         2|     samsung|01-05-2023| 980000|    3860000| 1286666.67|
|         2|     samsung|01-06-2023|1100000|    3880000| 1293333.33|
|         3|     oneplus|01-03-2023|1160000|    3380000| 1126666.67|
|         3|     oneplus|01-04-2023|1170000|    3450000|  1150000.0|
|         3|     oneplus|01-05-2023|1175000|    3505000| 1168333.33|
|         3|     oneplus|01-06-202

####Nested Json

In [0]:
restaurant_json_data = spark.read.format('json')\
    .option('multiline','true')\
    .option('inferschema','true')\
    .load('/FileStore/tables/resturant_json_data.json')

In [0]:
restaurant_json_data.show()

+----+-------+--------------------+-------------+-------------+-------------+------+
|code|message|         restaurants|results_found|results_shown|results_start|status|
+----+-------+--------------------+-------------+-------------+-------------+------+
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17066603}, b9...|         6835|           20|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17093124}, b9...|         8680|           20|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17580142}, b9...|          943|           20|            1|  null|
|null|   null|                  []|            0|            0|  

In [0]:
restaurant_json_data.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
restaurant_json_data.select('*', explode('restaurants').alias('new_restaurant')).printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [0]:
restaurant_json_data.select('*', explode('restaurants').alias('new_restaurant'))\
    .drop('restaurants')\
    .select('*','new_restaurant.restaurant.R.res_id',
            explode('new_restaurant.restaurant.establishment_types').alias('est_types'),
            'new_restaurant.restaurant.name').printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- results_found: long (nullable = true)
 |-- results_shown: long (nullable = true)
 |-- results_start: string (nullable = true)
 |-- status: string (nullable = true)
 |-- new_restaurant: struct (nullable = true)
 |    |-- restaurant: struct (nullable = true)
 |    |    |-- R: struct (nullable = true)
 |    |    |    |-- res_id: long (nullable = true)
 |    |    |-- apikey: string (nullable = true)
 |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |-- cuisines: string (nullable = true)
 |    |    |-- currency: string (nullable = true)
 |    |    |-- deeplink: string (nullable = true)
 |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- events_url: string (nullable = true)
 |    |    |-- featured_image: string (nullable = true)
 |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |-- has_tabl

In [0]:
restaurant_json_data.select('*', explode('restaurants').alias('new_restaurant'))\
    .drop('restaurants')\
    .select('*','new_restaurant.restaurant.R.res_id',
            explode_outer('new_restaurant.restaurant.establishment_types').alias('est_types'),
            'new_restaurant.restaurant.name').drop('new_restaurant').show()

+----+-------+-------------+-------------+-------------+------+--------+---------+--------------------+
|code|message|results_found|results_shown|results_start|status|  res_id|est_types|                name|
+----+-------+-------------+-------------+-------------+------+--------+---------+--------------------+
|null|   null|         6835|           20|            1|  null|17066603|     null|            The Coop|
|null|   null|         6835|           20|            1|  null|17059541|     null|Maggiano's Little...|
|null|   null|         6835|           20|            1|  null|17064405|     null|Tako Cheena by Po...|
|null|   null|         6835|           20|            1|  null|17057797|     null|Bosphorous Turkis...|
|null|   null|         6835|           20|            1|  null|17057591|     null|Bahama Breeze Isl...|
|null|   null|         6835|           20|            1|  null|17064266|     null|Hawkers Asian Str...|
|null|   null|         6835|           20|            1|  null|1

In [0]:
restaurant_json_data.select('*', explode('restaurants').alias('new_restaurant'))\
    .drop('restaurants')\
    .select('*','new_restaurant.restaurant.R.res_id',
            explode_outer('new_restaurant.restaurant.establishment_types').alias('est_types'),
            'new_restaurant.restaurant.name').drop('new_restaurant','code','message','results_found','results_shown','results_start','status').show(truncate=False)

+--------+---------+------------------------------------+
|res_id  |est_types|name                                |
+--------+---------+------------------------------------+
|17066603|null     |The Coop                            |
|17059541|null     |Maggiano's Little Italy             |
|17064405|null     |Tako Cheena by Pom Pom              |
|17057797|null     |Bosphorous Turkish Cuisine          |
|17057591|null     |Bahama Breeze Island Grille         |
|17064266|null     |Hawkers Asian Street Fare           |
|17060516|null     |Seasons 52 Fresh Grill              |
|17060320|null     |Raglan Road Irish Pub and Restaurant|
|17059060|null     |Hillstone                           |
|17059012|null     |Hollerbach's Willow Tree Café       |
|17060869|null     |Texas de Brazil                     |
|17061231|null     |The Ravenous Pig                    |
|17058534|null     |Earl of Sandwich                    |
|17057925|null     |Café Tu Tu Tango                    |
|17064031|null